Here we generate samples for the $\phi^4$ model with a $g_6 \cdot \phi^6$ coupling and see how $g_6$ changes under RG

In [ ]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, Optim, GalacticOptim, Plots

# Define the Phi4Model structure for 2D, now with phi^6 term
struct Phi4Model2D
    lattice_size::Int    # Lattice size (NxN)
    lambda::Float64      # Quartic coupling constant
    m2::Float64          # Mass term squared
    g6::Float64          # Sextic coupling constant
end

# Function to calculate the total energy (kinetic + potential) of a 2D field configuration
function calculate_energy_2d(phi::Matrix{T}, model::Phi4Model2D) where T<:Real
    N = size(phi, 1)
    kinetic_energy = 0.0
    potential_energy = 0.0

    for i in 1:N, j in 1:N
        # kinetic term: discrete Laplacian (periodic)
        i_right = mod1(i + 1, N)
        j_up    = mod1(j + 1, N)
        lap = 4*phi[i,j] - phi[i_right, j] - phi[mod1(i-1,N), j] - phi[i, j_up] - phi[i, mod1(j-1,N)]
        kinetic_energy += 0.5 * lap * phi[i,j]

        # potential term: mass, quartic and sextic interactions
        potential_energy += 0.5 * model.m2 * phi[i,j]^2
        potential_energy += 0.25 * model.lambda * phi[i,j]^4
        potential_energy += (1/6) * model.g6 * phi[i,j]^6
    end

    return kinetic_energy + potential_energy
end

# Define the target density for HMC (now includes g6)
struct Phi4TargetDensity2D
    model::Phi4Model2D
end

function LogDensityProblems.logdensity(p::Phi4TargetDensity2D, phi_vec::Vector{T}) where T<:Real
    N = p.model.lattice_size
    phi = reshape(phi_vec, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::Phi4TargetDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{Phi4TargetDensity2D}) = LogDensityProblems.LogDensityOrder{1}()

# HMC data generation (unchanged kinetic form)
function generate_phi4_data_hmc_2d(model::Phi4Model2D, n_samples::Int, n_adapts::Int=2000)
    N = model.lattice_size
    target = Phi4TargetDensity2D(model)
    initial = randn(N, N)

    metric     = DiagEuclideanMetric(N^2)
    hamiltonian= Hamiltonian(metric, target, ForwardDiff)
    eps        = find_good_stepsize(hamiltonian, vec(initial))
    integrator = Leapfrog(eps)
    kernel     = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor    = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    samples_vec, _ = sample(hamiltonian, kernel, vec(initial), n_samples, adaptor, n_adapts; progress=false)
    return [reshape(s, N, N) for s in samples_vec]
end

# Score matching objective with phi^6, no spacing parameter
function score_matching_objective_2d(lambda::Float64, m2::Float64, g6::Float64, data::Vector{Matrix{Float64}})
    N = size(data[1],1)
    obj = 0.0
    for phi in data
        for i in 1:N, j in 1:N
            x = phi[i,j]
            # discrete Laplacian for gradient
            xL = phi[mod1(i-1,N), j]; xR = phi[mod1(i+1,N), j]
            xD = phi[i, mod1(j-1,N)]; xU = phi[i, mod1(j+1,N)]
            kinetic = -(4*x - xL - xR - xU - xD)

            # full score: kinetic - potential gradient
            score = kinetic - (m2*x + lambda*x^3 + g6*x^5)
            # derivative of score wrt x
            ds = -(m2 + 3*lambda*x^2 + 5*g6*x^4 + 4)

            obj += 0.5*score^2 + ds
        end
    end
    return obj/length(data)
end

# Estimate parameters [λ, m2, g6] via particle swarm
function estimate_parameters_score_matching_2d(data)
    f(x) = score_matching_objective_2d(x[1], x[2], x[3], data)
    initial = [0.1, 0.1, 0.1]
    upper = [10.0, 10.0, 10.0]
    lower = [0.0, -10.0, 1e-5]
    swarm = ParticleSwarm(upper=upper, lower=lower)
    res   = optimize(f, initial, swarm)
    vals = res.minimizer
    println("Estimated λ=$(vals[1]), m2=$(vals[2]), g6=$(vals[3])")
    return vals[1],vals[2],vals[3]
end

# Block spin transform: median + kinetic renormalization
function compute_raw_grad_squared(phi::Matrix{Float64})
    gradx = circshift(phi, (-1, 0)) .- phi
    grady = circshift(phi, (0, -1)) .- phi
    return mean(gradx.^2 .+ grady.^2)
end

#Now we compute the blocked field and normalize it to match the kinetic term
function block_spin_transform_with_normalization(phi::Matrix{Float64})
    nx, ny = size(phi)
    nx_new, ny_new = div(nx, 2), div(ny, 2)
    phi_blocked = zeros(Float64, nx_new, ny_new)
    for i in 1:nx_new, j in 1:ny_new
        phi_blocked[i, j] = median(vec(phi[2i-1:2i, 2j-1:2j]))
    end
    raw_grad_old = compute_raw_grad_squared(phi)
    raw_grad_new = compute_raw_grad_squared(phi_blocked)
    Z_phi = raw_grad_new / raw_grad_old
    phi_blocked_normalized = phi_blocked ./ sqrt(Z_phi)
    return phi_blocked_normalized
end


function block_spin_samples(samples::Vector{Matrix{Float64}})
    return [block_spin_transform_with_normalization(sample) for sample in samples]
end

# Run RG procedure with parameter estimation at each step
function run_rg_procedure_phi4(model::Phi4Model2D, samples::Vector{Matrix{Float64}}, n_steps::Int, reps::Int)
    lambdas = Float64[]; m2s = Float64[]; g6s = Float64[]; sizes = Int[]
    push!(lambdas, model.lambda); push!(m2s, model.m2); push!(g6s, model.g6)
    push!(sizes, size(samples[1],1))
    current = samples
    for step in 1:n_steps
        current = block_spin_samples(current)
        sz = size(current[1],1)
        est_l = Float64[]; est_m2 = Float64[]; est_g6 = Float64[]
        for rep in 1:reps
            l,m2,g6 = estimate_parameters_score_matching_2d(current)
            push!(est_l, l); push!(est_m2, m2); push!(est_g6, g6)
        end
        push!(lambdas, mean(est_l)); push!(m2s, mean(est_m2)); push!(g6s, mean(est_g6)); push!(sizes, sz)
    end
    return lambdas, m2s, g6s, sizes
end


# Example usage:
lattice_size = 8
true_model = Phi4Model2D(lattice_size, 0.5, -0.2, 0.1)
n_samples  = 10000
samples = generate_phi4_data_hmc_2d(true_model, n_samples)
λ, m2, g6 = estimate_parameters_score_matching_2d(samples)
println("Results: λ=$(λ), m2=$(m2), g6=$(g6)")
